In [6]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import itertools as itr
import scipy.spatial.distance as ssd
import Objective_Value as OB

print("DYNAMIC TABU LIST WITH ASPIRATION \
BEST SO FAR and MULTI-RESTART WITH MUTATION ")

DYNAMIC TABU LIST WITH ASPIRATION BEST SO FAR and MULTI-RESTART WITH MUTATION 


In [7]:
# Data
# X coordinates
x_coordinates = np.array([82,82,82,82,82,82,82,82,82,82,\
                            96,50,49,13,29,58,84,14,2,3,\
                            5,98,84,61,1,88,91,19,93,50,\
                            98,5,42,61,9,80,57,23,20,85,98])

# Y coordinates
y_coordinates = np.array([76,76,76,76,76,76,76,76,76,76,\
                            44,5,8,7,89,30,39,24,39,82,\
                            10,52,25,59,65,51,2,32,3,93,\
                            14,42,9,62,97,55,69,15,70,60,5])

capacities = np.array([0,0,0,0,0,0,0,0,0,0,\
                       19,21,6,19,7,12,16,6,16,8,\
                       14,21,16,3,22,18,19,1,24,8,\
                       12,4,8,24,24,2,20,15,2,14,9])

In [8]:
# Stack them beside each other
x_y_coordinates = np.column_stack((x_coordinates,\
                                     y_coordinates))

# Create a distance matrix from the Euclidean distance
distance_matrix = ssd.cdist(x_y_coordinates,\
                            x_y_coordinates,'euclidean')

# Create a distance dataframe
dist_dataframe = pd.DataFrame(distance_matrix,\
                              columns=[i for i in range(1,42,1)],\
                              index=[i for i in range(1,42,1)])
# Create a capacity dataframe
cap_dataframe = pd.DataFrame(capacities,\
                             columns=["Capacity"],\
                             index=[i for i in range(1,42,1)])

In [9]:
# Initial solution to start with
x_0 = [24,26,22,31,11,16,25,30,34,12,14,1,20,39,15,5,36,13,18,19,32\
       ,38,28,10,9,23,33,21,7,4,40,35,2,17,8,6,37,3,29,27,41]

In [10]:
### VARIABLES ##
Runs = 2500
P_Mutation = 0.2

### TABU LIST ###
Length_of_Tabu_List = 20

Tabu_List = np.empty((0,len(x_0)+1)) # +1 to keep track of the fitness value

Final_Solution = []


# Initial random solution
rnd_sol_1 = x_0[:]


Total_Distance_Initial = OB.Complete_Distance_Not_Random(rnd_sol_1) # Get the distance of the initial solution

print("Initial Distance:", Total_Distance_Initial) # Print the initial distance
print("Initial Random Solution:", rnd_sol_1) # Print the initial solution

Initial Distance: 1558.2869666117285
Initial Random Solution: [24, 26, 22, 31, 11, 16, 25, 30, 34, 12, 14, 1, 20, 39, 15, 5, 36, 13, 18, 19, 32, 38, 28, 10, 9, 23, 33, 21, 7, 4, 40, 35, 2, 17, 8, 6, 37, 3, 29, 27, 41]


In [11]:
saved_solutions = np.empty((0,len(x_0)+1))

Index_For_Best_1 = np.empty((0,2))

Index_For_Best_2 = np.empty((0,2))


best_solution = []

Iterations = 1

for i in [0]:
    
    print()
    print("--> This is the %i" % Iterations, "th Iteration <--")
    
    swap_combinations = list(itr.combinations(rnd_sol_1, 2))
    value_array = np.empty((0,len(x_0)+1))
    
#    current_best = float('inf')
    for each in swap_combinations:
        x_swap = []
        each = list(each)
        

        for j in rnd_sol_1:

            if j == each[0]: 
                x_swap.append(each[1])

            elif j == each[1]:
                x_swap.append(each[0])

            else:
                x_swap.append(j)
        
        cost = OB.Complete_Distance_Not_Random(x_swap)
        
        penalty_1 = OB.Penalty_Capacity_Violation(x_swap,20,5)
        penalty_2 = OB.Penalty_Sequence(x_swap,200,300,5)
        penalty_3 = OB.Penalty_End_Points(x_swap,400,5)

        total_value = cost + penalty_1 + penalty_2 + penalty_3 # Add the costs to the penalties
        x_swap.insert(0,total_value)
        value_array = np.vstack((value_array,np.asarray(x_swap)))
    
    value_array_ordered = np.array(sorted(value_array, key=lambda x: x[0]))
    current_solution = value_array_ordered[0]
    
    if len(saved_solutions) >= 1:
        
        best_solution = saved_solutions[np.argmin(saved_solutions, axis=0)[0]]

        if current_solution in Tabu_List:
            t = 0
            while current_solution[0] in Tabu_List[:,0]:
                if current_solution[0] < best_solution[0]:
                    current_solution = best_solution
                    break
                else:
                    current_solution = value_array_ordered[t]
                    t = t+1
        else:
            current_solution = best_solution
          
        if len(Tabu_List) >= Length_of_Tabu_List:
            
            if Tabu_List[0,0] != current_solution[0]:
                Difference = len(Tabu_List) - Length_of_Tabu_List
                Diff = (Difference+1)
                for i in range(Diff):
                    Tabu_List = np.delete(Tabu_List, (Length_of_Tabu_List-1), axis=0)
                    Tabu_List = np.vstack((current_solution,Tabu_List))
                
            else:
                Tabu_List[0,:] = current_solution[np.newaxis]
        else:
            Tabu_List = np.vstack((current_solution,Tabu_List))
    
    else:
        pass

    
    
    # all the solutions evaluated are stored in "saved_solution"
    saved_solutions = np.vstack((current_solution,saved_solutions)) 
    
    Index_For_Best_1 = np.hstack((current_solution[0],Iterations)) 
    Index_For_Best_2 = np.vstack((Index_For_Best_1,Index_For_Best_2)) 


--> This is the 1 th Iteration <--


In [ ]:
    Mod_Iterations_1 = Iterations%20 
    Mod_Iterations_2 = Iterations%40 
    Mod_Iterations_3 = Iterations%30 
    
    if Mod_Iterations_3 == 0: # To return to the best so far in case you kept getting worse solutions
        if current_solution[0] > best_solution[0]: # If where you are is worse than the best so far
            current_solution = best_solution # Become best so far
            print("Go Back to best_solution")
        else:
            pass
    
    Random_1 = np.random.rand() # Take a random number to mutate
    
    if Random_1 <= P_Mutation: # Probablity is less than the probablity of mutation
        '''
        if Mod_Iterations == 0:
        '''   
        Mutated_current_solution = []
        
        Ran_Mut_1 = np.random.rand() # Probablity to Mutate
        Ran_Mut_2 = np.random.randint(1,len(x_0)+1) # Random integer in the length of the solution
        Ran_Mut_3 = np.random.randint(1,len(x_0)+1)

        while Ran_Mut_2 == Ran_Mut_3:
            Ran_Mut_3 = np.random.randint(1,len(x_0)+1) # If A1 is equal to A2, repeat
        
        
        if Ran_Mut_2 < Ran_Mut_3: # If A1 comes before A2
            M_Child_1_Pos_1 = current_solution[Ran_Mut_2] # Index A1 in the solution
            M_Child_1_Pos_2 = current_solution[Ran_Mut_3] # Index A2 in the solultion
            Ran_Mut_3 = Ran_Mut_3+1
            Rev_1 = current_solution[:]
            Rev_2 = list(reversed(current_solution[Ran_Mut_2:Ran_Mut_3])) # Reverse the order
            t = 0
            for i in range(Ran_Mut_2,Ran_Mut_3):
                Rev_1[i] = Rev_2[t]
                t = t+1
            
            Mutated_current_solution = Rev_1[:]
        
        else:
            M_Child_1_Pos_1 = current_solution[Ran_Mut_3] # Index A2 in the solution
            M_Child_1_Pos_2 = current_solution[Ran_Mut_2] # Index A1 in the solution
            Ran_Mut_2 = Ran_Mut_2+1
            Rev_1 = current_solution[:]
            Rev_2 = list(reversed(current_solution[Ran_Mut_3:Ran_Mut_2])) # Reverse the order
            t = 0
            for i in range(Ran_Mut_3,Ran_Mut_2):
                Rev_1[i] = Rev_2[t]
                t = t+1
            
            Mutated_current_solution = Rev_1[:]
        
       
        Ran_1 = np.random.randint(1,len(x_0)+1) # Random integers in length of solution
        Ran_2 = np.random.randint(1,len(x_0)+1)
        Ran_3 = np.random.randint(1,len(x_0)+1)
        Ran_4 = np.random.randint(1,len(x_0)+1)
        
        
        Xt = []
        A1 = Mutated_current_solution[Ran_1] # Get a random integer in the mutated solution
        A2 = Mutated_current_solution[Ran_2] # Get a random integer in the mutated solution
        
        # Making a new list of the new set of nodes
        T_Temp = Mutated_current_solution
        
        w = 0
        for i in T_Temp:
            if T_Temp[w] == A1:
                Xt=np.append(Xt,A2)
            elif T_Temp[w] == A2:
                Xt=np.append(Xt,A1)
            else:
                Xt=np.append(Xt,T_Temp[w])
            w = w+1
        
        Mutated_current_solution = Xt
        
        # Same nodes gets switched
        
        Xt = []
        A1 = Mutated_current_solution[Ran_1]
        A2 = Mutated_current_solution[Ran_3]
        
        # Making a new list of the new set of nodes
        w = 0
        for i in Mutated_current_solution:
            if Mutated_current_solution[w] == A1:
                Xt=np.append(Xt,A2)
            elif Mutated_current_solution[w] == A2:
                Xt=np.append(Xt,A1)
            else:
                Xt=np.append(Xt,Mutated_current_solution[w])
            w = w+1
        
        current_solution = Xt
        
        
        
    rnd_sol_1 = current_solution[1:]

    
    # Aspiration Criteria, Diversification, to Encourage the Search to Diversify
    
    if Mod_Iterations_2 == 0:
        Ran_1 = np.random.randint(1,len(x_0)+1) # Get a random integer
        Ran_2 = np.random.randint(1,len(x_0)+1)
        best_solution[Ran_1],best_solution[Ran_2] = best_solution[Ran_2],best_solution[Ran_1]
        Ran_1 = np.random.randint(1,len(x_0)+1)
        best_solution[Ran_1],best_solution[Ran_2] = best_solution[Ran_2],best_solution[Ran_1]
        Ran_3 = np.random.randint(1,len(x_0)+1)
        Ran_4 = np.random.randint(1,len(x_0)+1)
        best_solution[Ran_3],best_solution[Ran_4] = best_solution[Ran_4],best_solution[Ran_3]
        Ran_3 = np.random.randint(1,len(x_0)+1)
        best_solution[Ran_3],best_solution[Ran_4] = best_solution[Ran_4],best_solution[Ran_3]
        rnd_sol_1 = best_solution[1:]
    
    '''
    print()
    print("New current_solution:", current_solution)
    '''
    Iterations = Iterations+1

    # Change length of Tabu List every 20 runs, between 5 and 25
    if Mod_Iterations_1 == 0:
        Length_of_Tabu_List = np.random.randint(10,26)
        '''
        print()
        print("Length of Tabu List New:",Length_of_Tabu_List)
        '''


t = 0
Final_Here = []
for i in saved_solutions:
    
    if (saved_solutions[t,0]) <= min(saved_solutions[:,0]):
        Final_2 = []
        Final_2 = [saved_solutions[t,0]]
        Final_Here = saved_solutions[t,:]
    t = t+1
        
A_2_Final = min(saved_solutions[:,0])

Final_Solution = Final_Here[np.newaxis]


t = 0
Index_Here = []
for i in Index_For_Best_2:
    
    if (Index_For_Best_2[t,0]) <= min(Index_For_Best_2[:,0]):
        Final_3 = []
        Final_3 = [Index_For_Best_2[t,0]]
        Index_Here = Index_For_Best_2[t,:]
    t = t+1
        
A_3_Final = min(Index_For_Best_2[:,0])

Index_for_Best = Index_Here[1]


# Show truck capacities
Temp_Cap = Final_Solution[0,1:]
To_Save_Capacities = []
t = 0
for j in Temp_Cap:
    if t < 41:
        To_Save_Capacities = np.append(To_Save_Capacities,Temp_Cap[t])
        t = t+1
'''
print()
print("Cities:",To_Save_Capacities)
'''
Truck_Cap_1_M_1,Truck_Cap_2_M_1 = [],[]    
for j in To_Save_Capacities: # Only the values between 1 and 10 (warehouses)
    if 1 <= j <= 10:
        Truck_Cap_1_M_1 = np.append(Truck_Cap_1_M_1,j)
for i in Truck_Cap_1_M_1: # The warehouses' indexes
    value_index = np.where(To_Save_Capacities == i)
    Truck_Cap_2_M_1 = np.append(Truck_Cap_2_M_1,value_index)
t = 0
Caps_All, Caps_Sum_One_By_One  = [],[]
for k in range(len(Truck_Cap_2_M_1)):
    if t < 9:
        Ind_1 = int(Truck_Cap_2_M_1[t])+1
        Ind_2 = int(Truck_Cap_2_M_1[t+1])
        if Ind_1 in range(11) and Ind_2 in range(11):
            Truck_Cap_3_M_1_1 = 0
        Truck_Cap_3_M_1_1 = To_Save_Capacities[Ind_1:Ind_2]
        u = 0
        '''
        print()
        print("Cities between Two WH:",Truck_Cap_3_M_1_1)
        '''
        Caps_One_By_One = []
        if len(Truck_Cap_3_M_1_1) != 0:
            for o in Truck_Cap_3_M_1_1:
                Caps_Sum = 0
                Capacity_1 = Truck_Cap_3_M_1_1[u]
                Capacity_2 = Cap_Dataframe.loc[Capacity_1,'Capacity']
                Caps_One_By_One = np.append(Caps_One_By_One,Capacity_2)
                Caps_All = np.append(Caps_All,Capacity_2)
                u = u+1
                Caps_Sum = sum(Caps_One_By_One)
            Caps_Sum_One_By_One = np.append(Caps_Sum_One_By_One,Caps_Sum)
    t = t+1


Without_Penalty = Final_Solution[:,1:].tolist()

Without_Penalty_Cost = OB.Complete_Distance_Not_Random(Without_Penalty[0])


print()
print()
print("DYNAMIC TABU LIST WITH ASPIRATION BEST SO FAR and MULTI-RESTART WITH RANDOM NUMBER FOR MUTATION")
print()
print("Initial Solution:",Initial_For_Final)
print("Initial Distance:", Total_Distance_Initial)
print()
print("Min in all Iterations:",Final_Solution[:,1:])
print()
print("The Lowest Distance is:",Final_Solution[:,0])
print("The Lowest Distance Without Penalties is:",Without_Penalty_Cost)
print()
print("Trucks' Capacities:",Caps_Sum_One_By_One)
print()
print("At Iteration #:",Index_for_Best)



To_Plot = np.flip(saved_solutions, 0)


Look = (Final_Solution[:,0]).tolist()
Look = float(Look[0])
Look = int(Look)

Without_Penalty_Cost = int(Without_Penalty_Cost)

plt.plot(To_Plot[:,0])
plt.axhline(y=Look,color="r",linestyle='--')
plt.title("Total Distance Traveled by Trucks",fontsize=20,fontweight='bold')
plt.xlabel("Iterations",fontsize=18,fontweight='bold')
plt.ylabel("Distance (KM)",fontsize=18,fontweight='bold')
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')
xyz=(Iterations/2.6, Look)
xyzz = (Iterations/2.5, Look+200)
plt.annotate("Minimum Reached at: %s" % Without_Penalty_Cost, xy=xyz, xytext=xyzz,
             arrowprops=dict(facecolor='black', shrink=0.001,width=1,headwidth=5),
             fontsize=12,fontweight='bold')
plt.show()
